# NRM python upstream client library tutorial

This tutorial covers the use of NRM's python upstream client library, in the context of running an external resource management strategy. Its cell's output are deterministic, and the executed version that is vendored in the source tree is checked by the project's CI, so its behavior should always be up-to-date with the latest version of the software, and no cells should be throwing exceptions.  

The next few cells are for setup purposes.

In [1]:
%%capture
cd ..

In [2]:
%load_ext nb_black

<IPython.core.display.Javascript object>

## Setup

This notebook uses `nrm`'s python library bindings and needs the `nrmd` daemon in the `$PATH`. 

Assuming the project is cloned **with submodules updated** (and the code unmodified), one needs to run the following from the root of the project before running it:

In [3]:
%%capture
%%bash
./shake.sh build # for the daemon 
./shake.sh client # for the upstream client
./shake.sh pyclient # for the shared client library

<IPython.core.display.Javascript object>

This allows to use the command-line daemon and client in the working directory, should you need to for debugging purposes. Those should be available in the jupyter-lab terminal.

In [4]:
%%bash
nrmd --help

nrmd

Usage: <unknown> [-i|--stdin] [CONFIG] [-y|--yaml]
  NRM Daemon

Available options:
  -h,--help                Show this help text
  -i,--stdin               Read configuration on stdin.
  CONFIG                   Input configuration with .yml/.yaml/.dh/.dhall
                           extension. Leave void for stdin (dhall) input.
  -y,--yaml                Assume configuration to be yaml(json is valid yaml)
                           instead of dhall.
  -h,--help                Show this help text


<IPython.core.display.Javascript object>

In [5]:
%%bash
nrm --help

nrm

Usage: nrm COMMAND
  NRM Client

Available options:
  -h,--help                Show this help text
  COMMAND                  Choice of operation.

Available commands:
  run                      Run the application via NRM
  killcmd                  Kill cmd
  killslice                Kill slice
  setpower                 Set power limit
  cpd                      Show current CPD
  list                     List existing slices
  state                    Show NRM state
  config                   Show NRM configuration
  listen-raw               Listen to raw NRM upstream pub messages
  listen-cpd               Listen to CPD messages
  listen-all               Listen to all upstream pub messages


<IPython.core.display.Javascript object>

## Defining experiments

Now that the daemon is properly set-up, we will configure and run some experiments using the python interface.

The next cell imports the upstream client library:

In [6]:
import nrm.tooling

<IPython.core.display.Javascript object>

This notebook will start `nrmd` on the same machine as the notebok, but the same interface should be available for remote execution:

In [15]:
host = nrm.tooling.Local()
# host=Remote( target="cc@129.114.108.201")

<IPython.core.display.Javascript object>

Note that the two classes (`Local` and `Remote`) offer the same methods to start nrmd and interact via blocking message passing primitives. The supported methods are the following:

In [8]:
import inspect

for a, x in inspect.getmembers(host, predicate=inspect.ismethod):
    print("%s: %s" % (a, x.__doc__))

__init__: None
check_daemon:  checks if nrmd is alive 
get_cpd:  Obtain the current Control Problem Description 
get_state:  Obtain the current daemon state 
run_workload:  Runs a workload via NRM. The `nrmd` daemon must be running. 
start_daemon:  start nrmd 
stop_daemon:  stops nrmd 
workload_exit_status:  Check the workload's exit status. 
workload_finished:  Checks NRM to see whether all tasks are finished. 
workload_recv:  Receive a message from NRM's upstream API. 
workload_send:  Send a message to NRM's upstream API. 


<IPython.core.display.Javascript object>

The next cell defines some node experimental node (`nrmd`) daemon configurations and some workloads. Please see notebook [notebooks/configuration.ipynb](notebooks/configuration.ipynb) for an overview of `nrmd`'s and the "Manifest"'s configuration format.

In [13]:
daemonCfgs = {
    "redirected_log": {"logfile": "/tmp/logfile_experiment2", "verbose": "Info"},
    # "other":your code
}

workloads = {
    "dummy":[{"cmd":"sleep",
              "args": ["60"] ,
              "sliceID": "toto",
              "manifest":{
                  "app": { "slice": {
                                 "cpus": 1,
                                 "mems": 1 
                           },  
                           "perfwrapper":{"perfLimit":{"fromOps":100000},"perfFreq":{"fromHz":1}}
                         },
                          "name": "perfwrap",
                         }
            }
            ],
    #"other":your code
}

{'dummy': [{'cmd': 'sleep', 'args': ['60'], 'sliceID': 'toto', 'manifest': {'app': {'slice': {'cpus': 1, 'mems': 1}, 'perfwrapper': {'perfLimit': {'fromOps': 100000}, 'perfFreq': {'fromHz': 1}}}, 'name': 'perfwrap'}}]}


<IPython.core.display.Javascript object>

The next cell defines some experiments:

In [10]:
experiments = {
    "example": (daemonCfgs["redirected_log"], workloads["dummy"]),
    # "other": your code
}

<IPython.core.display.Javascript object>

## Running experiments
The next cell shows how to start the daemon and obtain the current control description problem.

In [16]:
host.start_daemon(daemonCfgs["redirected_log"])
assert host.check_daemon()
cpd=host.get_cpd()
print("The control problem description is:\n %s" % cpd)

connecting
connected to tcp://localhost:2345
The control problem description is:
 Problem 
    { sensors = fromList 
        [ 
            ( SensorID { sensorID = "RaplKey (PackageID 0)" }
            , Sensor 
                { range = 0.0 ... 2.6214332885e11
                , maxFrequency = 3.0
                } 
            ) 
        ]
    , actuators = fromList 
        [ 
            ( ActuatorID { actuatorID = "RaplKey (PackageID 0)" }
            , Actuator 
                { actions = 
                    [ DiscreteDouble 180.0
                    , DiscreteDouble 200.0
                    ] 
                }
            ) 
        ]
    , objectives = []
    , constraints = []
    } 


<IPython.core.display.Javascript object>

In [12]:
host.stop_daemon()
assert host.check_daemon() == False

<IPython.core.display.Javascript object>

### Dummy experiment: printing measurements

We now are ready to run an external resource management strategy. Using the low-level message passing interface:

In [20]:
import json
for name, (daemonCfg, workload) in experiments.items():
    # starting the daemon (does act as silent restart)
    host.start_daemon(daemonCfg)
    print("Starting the workload")
    host.run_workload(workload)
    print("Operating with the following problem:")
    print(host.get_cpd())
    # message passting exchange:
    while host.check_daemon() and not host.workload_finished():
        measurement_message = host.workload_recv()
        msg=json.loads(measurement_message)
        if "pubMeasurements" in msg:
            print("received raw data %s" % (str(msg)))
            
            
        command_message = "insert your code here"
        host.workload_send(command_message)
    print(host.check_daemon())

host.stop_daemon()

connecting
connected to tcp://localhost:2345
Starting the workload
Operating with the following problem:
Problem 
    { sensors = fromList 
        [ 
            ( SensorID { sensorID = "RaplKey (PackageID 0)" }
            , Sensor 
                { range = 0.0 ... 2.6214332885e11
                , maxFrequency = 3.0
                } 
            ) 
        ]
    , actuators = fromList 
        [ 
            ( ActuatorID { actuatorID = "RaplKey (PackageID 0)" }
            , Actuator 
                { actions = 
                    [ DiscreteDouble 180.0
                    , DiscreteDouble 200.0
                    ] 
                }
            ) 
        ]
    , objectives = []
    , constraints = []
    } 
receiving
{'pubMeasurements': [1579806163155595, [{'time': 1579806163155595, 'sensorValue': 148802207322, 'sensorID': {'sensorID': 'RaplKey (PackageID 0)'}}]]}
receiving
{'pubMeasurements': [1579806164155466.8, [{'time': 1579806164155466.8, 'sensorValue': 148812892780, 's

<IPython.core.display.Javascript object>

((0, False, False), 2345, 3456, 'localhost')

<IPython.core.display.Javascript object>

In [30]:
nrm.tooling.lib.pubAddress(host.commonOpts)

'tcp://localhost:2345'

<IPython.core.display.Javascript object>